In [7]:
import tweepy
import pandas as pd
import time

# Authentication with Twitter API V2
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAADcsxQEAAAAA1FJXhrBL%2FXo3Nlw78YC5TVO4jns%3D30eM1mRWzb9erLFlvSrqupvdS9pByR6xE530Vn6gJmK26ychGb"

# Set up Tweepy client for API V2
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Define query groups with Kosovo and Serbia conflict terms
query_group = (
    '"Kosovo" AND "Serbia" AND ("conflict" OR "war" OR "genocide" OR "ethnic cleansing" OR "independence" '
    'OR "NATO" OR "territory" OR "border dispute" OR "self-determination" OR "peace talks" OR "war crimes" '
    'OR "Kosovo War" OR "1999" OR "KLA" OR "SerbiaKosovo" OR "UN Resolution" OR "Kosovo Serbs" OR "ethnic violence" '
    'OR "Kosovo independence" OR "Kosovo conflict")'
)

# Initialize variables
tweets_data = []
tweet_count = 0
next_token = None
tweet_ids = set()  # Set to track unique tweet IDs
max_tweets = 100  # Set the total number of tweets you want to retrieve per query group

# Function to save data to CSV after each request
def save_to_csv(data, filename="kosovo_serbia_tweets.csv"):
    df = pd.DataFrame(data, columns=["Date", "User", "Tweet"])
    df.to_csv(filename, mode='a', header=not bool(tweet_count), index=False)
    print(f"Saved {len(data)} tweets to CSV.")

# Loop through the query group
while tweet_count < max_tweets:
    try:
        if next_token:
            tweets = client.search_recent_tweets(query=query_group, max_results=100, next_token=next_token)
        else:
            tweets = client.search_recent_tweets(query=query_group, max_results=100)

        # Extract tweet details and store in list
        for tweet in tweets.data:
            if tweet.id not in tweet_ids:  # Check if tweet ID is already in the set
                tweet_ids.add(tweet.id)  # Add the tweet ID to the set
                tweets_data.append([tweet.created_at, tweet.author_id, tweet.text])
                tweet_count += 1  # Increment the overall tweet count

        # Save to CSV after fetching the group of tweets
        save_to_csv(tweets_data)

        # Update next_token for pagination to ensure no duplicate results
        next_token = tweets.meta.get('next_token') if 'next_token' in tweets.meta else None

        # Break if no more tweets are available
        if not next_token or tweet_count >= max_tweets:
            break

    except tweepy.TooManyRequests as e:
        # Wait for the rate limit window to reset
        print("Rate limit exceeded, waiting for 15 minutes...")
        time.sleep(15 * 60)  # Wait for 15 minutes

    except Exception as e:
        print(f"An error occurred: {e}")
        break

print(f"Total tweets collected: {tweet_count}")


Rate limit exceeded, waiting for 15 minutes...


KeyboardInterrupt: 